# Slides descriptions

## Install and data

In [12]:
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

# Rutas de los archivos zip en Google Drive
test_zip_path = '/content/drive/MyDrive/test.zip'
dev_zip_path = '/content/drive/MyDrive/dev.zip'

# Directorio de destino en Colab
colab_content_path = '/content/'

# Copiar los archivos zip a Colab
!cp "{test_zip_path}" "{colab_content_path}"
!cp "{dev_zip_path}" "{colab_content_path}"

print(f"Archivos copiados a {colab_content_path}")
print(os.listdir(colab_content_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archivos copiados a /content/
['.config', 'drive', 'dev.lst', 'test', 'dev.zip', 'test.zip', 'sample_data']


In [13]:
!unzip /content/test.zip -d /content/test
!unzip /content/dev.zip -d /content/dev

Archive:  /content/test.zip
replace /content/test/test.lst? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  /content/dev.zip
   creating: /content/dev/500011/
   creating: /content/dev/550000/
   creating: /content/dev/592/
   creating: /content/dev/609/
   creating: /content/dev/624000/
  inflating: /content/dev/dev.lst    
   creating: /content/dev/609/manual_transcription/
   creating: /content/dev/609/manual_translations/
  inflating: /content/dev/609/slides.pptx  
  inflating: /content/dev/609/609.m4a  
   creating: /content/dev/609/manual_transcription/sentence_segmented/
  inflating: /content/dev/609/manual_transcription/ie609.srt  
  inflating: /content/dev/609/manual_transcription/ie609.txt  
  inflating: /content/dev/609/manual_transcription/sentence_segmented/ie609.srt  
   creating: /content/dev/609/manual_translations/de/
   creating: /content/dev/609/manual_translations/es/
   creating: /content/dev/609/manual_translations/fr/
   creating: /content/dev/609/manual_transl

In [3]:
! pip install  backoff  transformers==4.49

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 135.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.1
    Uninstalling transformers-4.55.1:
      Successfully uninstalled transformers-4.55.1


In [4]:
# ────────────────────────────── 1. Dependencias de SO ─────────────────────────
!apt-get update -qq
!apt-get install -y poppler-utils libreoffice

# ────────────────────────────── 2. Paquetes Python ────────────────────────────
!pip install pdf2image pillow tqdm   # pdf2image necesita Poppler

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor default-jre default-jre-headless dictionaries-common
  firebird3.0-common firebird3.0-common-doc firebird3.0-server-core
  firebird3.0-utils fonts-crosextra-caladea fonts-crosextra-carlito
  fonts-dejavu fonts-dejavu-core fonts-dejavu-extra fonts-liberation2
  fonts-linuxlibertine fonts-noto-core fonts-noto-extra fonts-noto-mono
  fonts-noto-ui-core fonts-opensymbol fonts-sil-gentium
  fonts-sil-gentium-basic gstreamer1.0-gl gstreamer1.0-gtk3 hunspell-en-us
  libabsl20210324 libabw-0.1-1 libatk-wrapper-java libatk-wrapper-java-jni
  libbsh-java libcdr-0.1-1 libclucene-contribs1v5 libclucene-core1v5
  libcolamd2 libe-book-0.1-1 libel-api-java 

## Load

### Load Test data

In [5]:


def get_files(folder='test'):
  with open(f'/content/{folder}/{folder}.lst', 'r', encoding='utf-8') as f:
    files = f.read()
    files = files.splitlines()
    print(files)
    return files

In [6]:
from pathlib import Path
import tempfile, subprocess
from typing import List, Tuple
from PIL import Image
from pdf2image import convert_from_path

def _ppt_to_pdf(src: Path) -> Path:
    tmp = Path(tempfile.mkdtemp())
    pdf_out = tmp / f"{src.stem}.pdf"
    cmd = ["soffice", "--headless", "--convert-to", "pdf", "--outdir", str(tmp), str(src)]
    subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return pdf_out  # se borrará cuando el tmp sea recolectado por el sistema

def _ensure_pdf(path: str | Path) -> Path:
    extensions = {".pdf", ".ppt", ".pptx"}
    ext = None
    for e in extensions:
        if Path(f"{path}{e}").exists():
            ext = e
            break
    p=Path(f"{path}{ext}")
    if ext == ".pdf":
        return p
    if ext in {".ppt", ".pptx"}:
        return _ppt_to_pdf(p)
    raise ValueError(f"Formato no soportado: {ext}. Usa .pdf/.ppt/.pptx")



def slides_to_images(file_path: str | Path, dpi: int = 150) -> List[Image.Image]:
    pdf = _ensure_pdf(file_path)
    return convert_from_path(pdf, dpi=dpi, fmt="png", output_folder=None)

In [11]:
from pathlib import Path
import datasets as ds


def get_all_images(folder="train", language="es", files=None,
                  dpi=150,
                  bg="white", store_slide_paths=False):
    rows = []
    files = get_files(folder) if files is None else files

    # Carga y parsea el maestro (una vez)

    for name in files:


        # 1) Preprocesa slides una vez (o usa caché si ya existen)
        slides_root = Path(f"/content/{folder}/{name}/slides")
        images=slides_to_images(slides_root, dpi=dpi)

        print(len(images))
        yield images



### Load Model

In [8]:
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

import torch

model_name = 'microsoft/Phi-4-multimodal-instruct'
processor = AutoProcessor.from_pretrained(model_name,trust_remote_code = True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    _attn_implementation='sdpa',
)

generation_config = GenerationConfig.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

processing_phi4mm.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-4-multimodal-instruct:
- processing_phi4mm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/image_processing_auto.py:594: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/root/.cache/huggingface/modules/transformers_modules/microsoft/Phi-4-multimodal-instruct/33e62acdd07cd7d6635badd529aa0a3467bb9c6a/speech_conformer_encoder.py:2774: FutureWarning: Please specify CheckpointImpl.NO_REENTRANT as CheckpointImpl.REENTRANT will soon be removed as the default and eventually deprecated.
  lambda i: encoder_checkpoint_wrapper(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Basic Inference

In [ ]:

from PIL import Image

device = torch.device("cuda")


def build_prompt():
    # Prompt de chat con placeholders multimodales
    task_text=f"""
    You are a teaching assistant. Describe the slide in 3 bullet points for a student:
    1) Title/topic
    2) 2–3 key ideas
    3) Visual elements and technical terminology.
    If there is no readable text, say: "Sin texto legible."
    Answer in Spanish, maximum 200 words.
    """
    messages = [
        {"role":"user",
         "content": "<|image_1|>\n" + task_text}
    ]
    return processor.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )



In [15]:
model.eval()
model.gradient_checkpointing_disable()   # evita warnings/bugs en eval
descriptions = []
prompt = build_prompt()
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    for images in get_all_images(folder='test',dpi=150, store_slide_paths=True):
      diap = []
      for i in images:

        inputs = processor(
          text=[prompt],
          images=[i],             # lista de PIL Images
          return_tensors="pt",
          padding=True
        ).to(device)
        gen_ids = model.generate(
            **inputs,
            max_new_tokens=500,
            do_sample=False,
            num_beams=1,
            use_cache=True,
            num_logits_to_keep=0
        )

        # cortar el prompt textual
        input_len = inputs["input_ids"].shape[1]
        decoded = processor.batch_decode(gen_ids[:, input_len:].cpu(), skip_special_tokens=True)
        diap.append(decoded[0])
      descriptions.append(diap)
      print(diap)

['404', '596001', '605000', '606', '545006']
55
['1) Título/tema: La diapositiva se titula "Welcome!" y se trata de una sesión de aprendizaje en línea que comienza a las 18:15 horas CET.\n2) 2–3 ideas clave: La diapositiva informa que la sesión es CME acreditada y que al final de la presentación, los participantes serán dirigidos a la evaluación y prueba de opción múltiple.\n3) Elementos visuales: La diapositiva contiene un logotipo en la esquina superior derecha, un cuadro de diálogo en la parte inferior con opciones de "Yes" y "No" para unirse a la conferencia, y un hashtag "#e_ESO" en la parte inferior.', '1) Título/tema: La diapositiva se titula "e-sesion 404" y discute el impacto de la cirugía oncológica en los resultados.\n2) 2–3 ideas clave: La diapositiva presenta a dos expertos, Prof. Charles M. Balch y Prof. Riccardo Audisio, que discuten este tema. También menciona que el contenido de la diapositiva se extrajo de la política de e-ESO, que proporciona nuevas habilidades y con

['1) Título/tema: La diapositiva se centra en cómo evaluar la calidad de los planes de tratamiento en radioterapia. Presenta a un experto y un disuciante que discuten este tema.\n\n2) 2-3 ideas clave: El experto, Dr. Núria Jorquet Sala, es del Prostate Institute Barcelona, Barcelona, España. El disuciante es Prof. Pierfrancesco Franco, de la University of Eastern Piedmont, Novara, Italia. La diapositiva discute la importancia de compartir experiencias en e-Eso y cómo usar el hashtag #e_ESO para hacerlo.\n\n3) Elementos visuales: La diapositiva no contiene gráficos, fórmulas o esquemas. Sin embargo, presenta el logo de e-Eso y el hashtag #e_ESO para compartir experiencias en e-Eso.', '1) Título/tema: La diapositiva se centra en cómo evaluar la calidad de los planes de tratamiento en radioterapia. Presenta la información de Nuria Jornet, una consultora médica física y cabeza del departamento de física de radioterapia en el Hospital de la Santa Creu i Sant Pau, Barcelona.\n\n2) 2-3 ideas 

['1) Título/tema: La diapositiva se centra en temas relacionados con el cáncer de mama avanzado y su impacto en la vida nocturna.\n2) 2–3 ideas clave: La diapositiva presenta información sobre expertos y discusantes en el tema, así como un extracto de la política de e-ESO. También incluye un llamado a compartir experiencias de e-ESO y un hashtag para ello.\n3) Elementos visuales: La diapositiva no contiene gráficos, fórmulas o esquemas.', '1) Título/tema: La diapositiva se centra en el tema de "Things about advanced breast cancer that keep us up at night", presentado por Hope S Rugo, MD, quien es un profesor de medicina y director de la educación en oncología y ensayos clínicos en la Universidad de California, San Francisco Medical Center.\n\n2) Ideas clave: La diapositiva probablemente discute los desafíos y preocupaciones asociados con el cáncer de mama avanzado, como el impacto emocional, los tratamientos disponibles y los resultados de los pacientes. También puede abordar los avanc

['1) Título/tema: La diapositiva se titula "Multidisciplinaria sesión on rectal cancer", lo que indica que el tema de la sesión es un enfoque multidisciplinario sobre el cáncer rectal.\n\n2) 2–3 ideas clave: La diapositiva presenta a tres expertos y dos participantes en la discusión, lo que sugiere que la sesión es un evento de discusión con múltiples perspectivas. También se menciona que la sesión es parte de las e-SES, lo que indica que es una sesión virtual.\n\n3) Elementos visuales: La diapositiva no contiene gráficos, fórmulas o esquemas, solo texto y logos.', '1) Título/tema: La diapositiva se centra en el tema de la gestión multidisciplinaria en el tratamiento de la enfermedad de Rectal Cancer. Presenta el nombre del orador, Katia Roque Pérez, M.D., y la institución asociada, Instituto Nacional de Enfermedades Neoplásicas Sociedad Peruana de Oncología Médica, ubicada en Lima, Perú.\n\n2) 2–3 ideas clave: La presentación probablemente discute los enfoques y estrategias para una g

In [17]:
descriptions

[['1) Título/tema: La diapositiva se titula "Welcome!" y se trata de una sesión de aprendizaje en línea que comienza a las 18:15 horas CET.\n2) 2–3 ideas clave: La diapositiva informa que la sesión es CME acreditada y que al final de la presentación, los participantes serán dirigidos a la evaluación y prueba de opción múltiple.\n3) Elementos visuales: La diapositiva contiene un logotipo en la esquina superior derecha, un cuadro de diálogo en la parte inferior con opciones de "Yes" y "No" para unirse a la conferencia, y un hashtag "#e_ESO" en la parte inferior.',
  '1) Título/tema: La diapositiva se titula "e-sesion 404" y discute el impacto de la cirugía oncológica en los resultados.\n2) 2–3 ideas clave: La diapositiva presenta a dos expertos, Prof. Charles M. Balch y Prof. Riccardo Audisio, que discuten este tema. También menciona que el contenido de la diapositiva se extrajo de la política de e-ESO, que proporciona nuevas habilidades y conocimientos para oncólogos y otros médicos int

In [21]:
model.eval()
model.gradient_checkpointing_disable()   # evita warnings/bugs en eval
descriptions_dev = []
prompt = build_prompt()
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    for images in get_all_images(folder='dev',dpi=150, store_slide_paths=True):
      diap = []
      for i in images:

        inputs = processor(
          text=[prompt],
          images=[i],             # lista de PIL Images
          return_tensors="pt",
          padding=True
        ).to(device)
        gen_ids = model.generate(
            **inputs,
            max_new_tokens=500,
            do_sample=False,
            num_beams=1,
            use_cache=True,
            num_logits_to_keep=0
        )
        i.close()


        # cortar el prompt textual
        input_len = inputs["input_ids"].shape[1]
        decoded = processor.batch_decode(gen_ids[:, input_len:].cpu(), skip_special_tokens=True)
        diap.append(decoded[0])

      descriptions_dev.append(diap)
      print(diap)
      del images, inputs, gen_ids
      torch.cuda.empty_cache()

['500011', '624000', '609', '550000', '592']
47
['1) Título/tema: La diapositiva se centra en el tema de "Controversias en la gestión de rectal cancer", que es un tema de salud relacionado con el cáncer rectal.\n\n2) 2–3 ideas clave: La diapositiva presenta la información sobre el profesor de Colorectal Surgery, Paris Tekkis, y su afiliación a Imperial College London, The Royal Marsden Hospital y Chelsea & Westminster Hospital. También menciona la fecha de la presentación, que es octubre de 2019.\n\n3) Elementos visuales: La diapositiva no contiene gráficos, fórmulas o esquemas. Sin embargo, presenta un logotipo de Imperial College London y un escudo de armas, que son elementos visuales que representan la institución.', '1) Título/tema: La diapositiva discute las consideraciones para la operación después de la quimioterapia pélvica, centrándose en el momento de la operación y la decisión de remover el prostato en casos avanzados de cáncer pélvico.\n\n2) 2–3 ideas clave: La diapositiva 

['1) Título/tema: La diapositiva se centra en el tema de "De-escalación de la local tratamiento para el cáncer de mama", que trata sobre la reducción de la intensidad del tratamiento local para el cáncer de mama.\n\n2) 2–3 ideas clave: La diapositiva presenta información sobre la de-escalación del tratamiento local para el cáncer de mama, incluyendo la presentación de un experto y un asistente. También discute la importancia de la verificación independiente de diagnósticos y dosis de medicamentos, y la necesidad de buscar consejo médico profesional.\n\n3) Elementos visuales: La diapositiva no contiene gráficos, fórmulas o esquemas. Sin embargo, presenta el logotipo de e-ESO y el hashtag #e_ESO para compartir experiencias de e-ESO.', '1) Título/tema: La diapositiva se centra en el tema de la de-escalación de la local tratamiento para el cáncer de mama. Se pregunta si es demasiado poco mejor que como mucho.\n\n2) 2-3 ideas clave: La diapositiva presenta la pregunta "¿Es demasiado poco me

['1) La diapositiva se titula "Radioterapia para cáncer de próstata: un relato histórico". Presenta a un experto y un discutidor, ambos con sus respectivos nombres y lugares de trabajo.\n2) La diapositiva incluye un extracto de la política de e-ESO, que explica que el sitio web proporciona presentaciones para profesionales de la oncología y otros interesados en la oncología. También menciona que los materiales son propiedad de los autores o e-ESO, y que e-ESO no es responsable de ningún daño o lesión.\n3) No hay elementos visuales como gráficos, fórmulas o esquemas en la diapositiva.', '1) Título/tema: La diapositiva se titula "Radioterapia para próstata cancer: an historical narrative review", lo que indica que el tema es la radioterapia para el cáncer de próstata y se tratará de manera histórica.\n\n2) 2–3 ideas clave: La diapositiva presenta la información de dos autores, Alberto Bossi, MD y Gustave Roussy Institute, Villejuif, Francia, y Amethyst Group, Paris, Francia. El tema se c

['1) Título/tema: TITAN: Thrombocytopenia\n2) 2-3 ideas clave: La diapositiva presenta la información sobre Thrombocytopenia, una condición médica. Presenta a dos expertos, Mr Johan De Munter y Mrs Ana Khmaladze, que discuten sobre este tema. También incluye un extracto de la política de e-ESO, que proporciona información sobre el uso de la plataforma e-ESO para la educación en oncología.\n3) Elementos visuales: La diapositiva no contiene gráficos, fórmulas o esquemas. Contiene solo texto y logos.', '1) Título/tema: TITAN - Programa de Educación y Entrenamiento para Enfermeras\n2) 2-3 ideas clave: El TITAN es un programa educativo diseñado para mejorar las habilidades y conocimientos de las enfermeras al tratar a pacientes con cáncer, trombocitopenia, anemia y neutropenia. El programa está disponible para miembros individuales o nacionales de EONS que deseen organizar un curso TITAN en su país.\n3) Elementos visuales: La diapositiva no contiene gráficos, fórmulas o esquemas.', "1) Títu

In [22]:
#
test=['404', '596001', '605000', '606', '545006']
dev=['500011', '624000', '609', '550000', '592']

dic={}
for i,j in zip(test,descriptions):
  dic[i]=j
for i,j in zip(dev,descriptions_dev):
  dic[i]=j

In [23]:
# save dict in drive
import pickle
with open('/content/drive/MyDrive/images_descriptions.pkl', 'wb') as f:
    pickle.dump(dic, f)